In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (10,7)

In [ ]:
df = pd.read_csv('../input/S&P.csv')
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
close = pd.Series(df.Close.values,index=pd.DatetimeIndex(df.Date))
close = close[::-1]

In [ ]:
close.plot(title='S&P 500 From Inception');

In [ ]:
close = close['1990-01-01':]

In [ ]:
returns = pd.Series(np.diff(close.values),index=close.index[1:])

In [ ]:
returns.plot();

In [ ]:
import pymc3 as pm
from pymc3.distributions.timeseries import GaussianRandomWalk

In [ ]:
with pm.Model() as model:
    step_size = pm.Exponential('sigma', 50.) #1
    s = GaussianRandomWalk('s', sd=step_size, #2
                           shape=len(returns))

    nu = pm.Exponential('nu', .1) #3

    r = pm.StudentT('r', nu=nu, #4
                    lam=pm.math.exp(-2*s),
                    observed=returns.values)

In [ ]:
with model:
    trace = pm.sample(tune=2000, nuts_kwargs=dict(target_accept=.9))

In [ ]:
pm.traceplot(trace, varnames=['sigma', 'nu']);

In [ ]:
plt.plot(returns.values)
plt.plot(np.exp(trace[s].T), 'r', alpha=.03);
plt.xlabel('time') 
plt.ylabel('returns')
plt.legend(['S&P500', 'Stochastic Vol.']);